RECUPERATION DE LA LISTE DE TOUS LES ETABLISSEMENTS (AVEC CHANGEMENTS DE PAGE)

In [ ]:
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

class SFAPSpider(scrapy.Spider):
    # Name of your spider
    name = "sfap"

    # Url to start your spider from
    start_urls = [
        'https://sfap.org/annuaire?contact_sub_type=Structure&type_6%5B%5D=USP&type_6%5B%5D=LISP'
    ]

    # Callback function that will be called when starting your spider
    def parse(self, response):
        # retourne une liste de tous les éléments de classe "quote" 
        entrees = response.css('tbody tr')
        for entree in entrees:
            nom = entree.css('td.views-field-display-name a::text').get()
            type = entree.css('td.views-field-type-6::text').get()
            adresse = entree.css('td.views-field-street-address::text').get()
            code_postal = entree.css('td.views-field-postal-code::text').get()
            commune = entree.css('td.views-field-city::text').get()
            telephone = entree.css('td.views-field-phone::text').get()
            yield {
                'nom': nom,
                'type':type.strip('\n').strip(),
                'adresse':adresse.strip('\n').strip(),
                'code_postal':code_postal.strip('\n').strip(),
                'commune':commune.strip('\n').strip(),
                'telephone':telephone.strip('\n').strip()
            }
        try:
            next_page = response.css('li.pager-next a').attrib['href']
        except:
            logging.info("No next page")
        else:
            yield response.follow(next_page, callback=self.parse)


filename = 'page_1.json'

if filename in os.listdir("src/"):
    os.remove('src/'+filename)

process = CrawlerProcess(settings={
    'USER_AGENT': 'Chrome/97.0',
    'LOG_LEVEL': logging.INFO,
    'FEEDS':{
        'src/' + filename : {"format": "json", 'encoding': 'utf8'},
        }
    "AUTOTHROTTLE_ENABLED": True
    }
)

#Lancer le crawler utilisant la spider
process.crawl(SFAPSpider)
process.start()

ON VA SUR CHAQUE PAGE POUR RECUPERER DES INFOS SUPPLEMENTAIRES

In [ ]:
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

class SFAPSpider(scrapy.Spider):
    # Name of your spider
    name = "sfap"

    # Url to start your spider from
    start_urls = [
        'https://sfap.org/annuaire?contact_sub_type=Structure&type_6%5B%5D=USP&type_6%5B%5D=LISP'
    ]

    # Callback function that will be called when starting your spider
    def parse(self, response):
        # retourne une liste de tous les éléments de classe "quote" 
        entrees = response.css('tbody tr')
        for entree in entrees:
            nom = entree.css('td.views-field-display-name a::text').get()
            url = entree.css('td.views-field-display-name a').attrib['href']
            type = entree.css('td.views-field-type-6::text').get()
            adresse = entree.css('td.views-field-street-address::text').get()
            code_postal = entree.css('td.views-field-postal-code::text').get()
            commune = entree.css('td.views-field-city::text').get()
            telephone = entree.css('td.views-field-phone::text').get()
            infos= {
                'nom': nom,
                'type':type.strip('\n').strip(),
                'adresse':adresse.strip('\n').strip(),
                'code_postal':code_postal.strip('\n').strip(),
                'commune':commune.strip('\n').strip(),
                'telephone':telephone.strip('\n').strip()
            }

            yield scrapy.Request("https://sfap.org"+url, callback=self.details, cb_kwargs = {'infos':infos})
                

        try:
            next_page = response.css('li.pager-next a').attrib['href']
        except:
            logging.info("No next page")
        else:
            yield response.follow(next_page, callback=self.parse)

    def details(self, response, infos):
        print("page ",infos['nom'])
        departement = response.css('#edit-contact0-address0-state-province-id > div:nth-child(2)::text').get()
        nb_lits = "A REMPLIR",#response.css('.form-item-contact:0-custom-13 > div:nth-child(2)::text').get()
        site_internet = "A REMPLIR"#response.css('div.form-item:nth-child(43) > div:nth-child(2) > a:nth-child(1)').get()
        zone_libre = response.css('#edit-contact0-custom-95::text').get()
        infos.update({'departement':departement, 'nb_lits':nb_lits,'zone_libre':zone_libre, 'site_internet':site_internet})

        yield infos

filename = 'toutes_les_pages.json'

if filename in os.listdir("src/"):
    os.remove('src/'+filename)

process = CrawlerProcess(settings={
    'USER_AGENT': 'Chrome/97.0',
    'LOG_LEVEL': logging.INFO,
    'FEEDS':{
        'src/' + filename : {"format": "json", 'encoding': 'utf8'},
        },
    #"AUTOTHROTTLE_ENABLED": True
    }
)

#Lancer le crawler utilisant la spider
process.crawl(SFAPSpider)
process.start()

INFOS DE PAGES PRECISES (ESSAIS)

In [1]:
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

class SFAPSpider(scrapy.Spider):
    # Name of your spider
    name = "sfap"

    # Url to start your spider from
    start_urls = [
        'https://sfap.org/annuaire/31728'
    ]

    # Callback function that will be called when starting your spider
    def parse(self, response):
        # retourne une liste de tous les éléments de classe "quote" 
        lignes = response.css('div.form-item')
        for ligne in lignes:
            label  = ligne.css('label::text').get()
            valeur = ligne.css('div div').get()
            
            yield {
                label: valeur
            }


filename = 'page_info.json'

if filename in os.listdir("src/"):
    os.remove('src/'+filename)

process = CrawlerProcess(settings={
    'USER_AGENT': 'Chrome/97.0',
    'LOG_LEVEL': logging.INFO,
    'FEEDS':{
        'src/' + filename : {"format": "json", 'encoding': 'utf8'},
        },
    "AUTOTHROTTLE_ENABLED": True
    }
)

#Lancer le crawler utilisant la spider
process.crawl(SFAPSpider)
process.start()

2024-11-16 13:05:59 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2024-11-16 13:05:59 [scrapy.utils.log] INFO: Versions: lxml 5.2.1.0, libxml2 2.10.4, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 24.0.0 (OpenSSL 3.0.15 3 Sep 2024), cryptography 42.0.5, Platform Windows-11-10.0.22631-SP0
2024-11-16 13:05:59 [scrapy.addons] INFO: Enabled addons:
[]
2024-11-16 13:05:59 [py.warnings] WARNING: c:\Users\Admin\anaconda3\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the docu